# Ejercicios 


1.  Encontrar la formulación  variacional de 

$$\begin{cases}-\nabla\cdot(\alpha\nabla u(x))=f(x), &\text{ con }x\in\Omega\\
u(x)=U_a,&\text{ con }x\in\Gamma_{D_a}\\
u(x)=U_b,&\text{ con }x\in\Gamma_{D_b}\\
\frac{\partial u}{\partial n}=0,&\text{ con }x\in\Gamma_{N}\end{cases}$$
 

**R/** Dado que se tienen condiciones de Dirichlet no homogeneas, defina:
 $$V \subseteq H^1(\Omega) = \{ v : || v || < \infty , ||\nabla v|| < \infty , v|_{\Gamma_{D_a}}=U_a, v|_{\Gamma_{D_b}}=U_b \}$$

Sea $u, v \in V$, integrando sobre $\Omega$:
$$- \int_{\Omega} v \Big( \nabla\cdot(\alpha\nabla u(x)) \Big) \, dx = \int_{\Omega} fv \, dx$$
Usando teorema de Green se obtiene que:
$$- \int_{\Omega} v \Big( \nabla\cdot(\alpha\nabla u(x)) \Big) \, dx = \int_{\Omega} \alpha \nabla u \cdot \nabla v \, dx - \int_{\partial \Omega} \alpha \frac{\partial u}{\partial \eta} v \, dS$$
Pero:
$$ - \int_{\partial \Omega} \alpha \frac{\partial u}{\partial \eta} v \, dS = - \Big( \int_{\Gamma_{D_a}} \alpha \frac{\partial u}{\partial \eta} v \, dS + \int_{\Gamma_{D_b}} \alpha \frac{\partial u}{\partial \eta} v \, dS + \int_{\Gamma_{N}} \alpha \frac{\partial u}{\partial \eta} v \, dS \Big) $$
Dado que $\frac{\partial u}{\partial \eta}=0, \text{ con }x\in\Gamma_{N}$ el ultimo termino se anula. Ademas, $u(x)=U_a$ en $\Gamma_{D_a}$ y $u(x)=U_b$ en $\Gamma_{D_b}$, entonces:
$$ - \int_{\partial \Omega} \alpha \frac{\partial u}{\partial \eta} v \, dS = - \int_{\Gamma_{D_a}} \alpha \frac{\partial U_a}{\partial \eta} v \, dS - \int_{\Gamma_{D_b}} \alpha \frac{\partial U_b}{\partial \eta} v \, dS  $$ 
Finalmente, se tiene que la forma variacional esta dada por:
$$ \int_{\Omega} \alpha \nabla u \cdot \nabla v \, dx - \int_{\Gamma_{D_a}} \alpha \frac{\partial U_a}{\partial \eta} v \, dS - \int_{\Gamma_{D_b}} \alpha \frac{\partial U_b}{\partial \eta} v \, dS = \int_{\Omega} f v \, dx $$
$$ \int_{\Omega} \alpha \nabla u \cdot \nabla v \, dx = \int_{\Gamma_{D_a}} \alpha \frac{\partial U_a}{\partial \eta} v \, dS + \int_{\Gamma_{D_b}} \alpha \frac{\partial U_b}{\partial \eta} v \, dS + \int_{\Omega} f v \, dx $$

2. Encontrar la formulación  variacional de
$$\begin{cases}-\nabla\cdot(\alpha(u)\nabla u(x))=f(x), &\text{ con }x\in\Omega\\
u(x)=g,&\text{ con }x\in\partial\Omega\end{cases}$$

**R/** Defina
 $$V \subseteq H^1(\Omega) = \{ v : || v || < \infty , ||\nabla v|| < \infty , v|_{\partial \Omega}= g \}$$
 y
 $$ \hat{V} \subseteq H^1_0(\Omega) = \{ v : || v || < \infty , ||\nabla v|| < \infty , v|_{\partial \Omega}= 0 \} $$

Sea $u \in V$ y $v \in \hat{V}$, integrando sobre $\Omega$:
$$ - \int_{\Omega} \Big( \nabla \cdot (\alpha(u) \nabla u(x) \Big) v \, dx = \int_{\Omega} f v \, dx $$
Usando teorema de Green se obtiene que:
$$- \int_{\Omega} \Big( \nabla \cdot (\alpha(u) \nabla u(x) \Big) v \, dx = \int_{\Omega} \alpha(u) \nabla u \cdot \nabla v \, dx - \int_{\partial \Omega} \alpha(u) \frac{\partial u}{\partial \eta} v \, dS$$
como $v$ se anula en $\partial \Omega$ el ultimo termino se anula, por lo tanto:
$$ \int_{\Omega} \alpha(u) \nabla u \cdot \nabla v \, dx =  \int_{\Omega} f v \, dx$$

3. Sea $\Omega=[0,1]\times[0,1]$ y $f(x,y)=x^2+y^2$. Suponiendo que
$$\Gamma_{D_a}=\{(x,y)\in\Omega:x=0\},\quad \Gamma_{D_b}=\{(x,y)\in\Omega:x=1\},$$
$$\quad \Gamma_{N}=\{(x,y)\in\Omega:y=0\}\cup\{(x,y)\in\Omega:y=1\}$$
y $U_a=1$, $U_b=1$, $\alpha=5$, Implementar la aproximación del problema 1  en Fenics y resolverlo con el método de Galerkin. Graficar la solución.

4. Sea $\Omega=[0,1]\times[0,1]$ y $f(x,y)=x^2+y^2$ y $\alpha(u)=u+3$. Implementar la aproximación del problema 2  en Fenics y resolverlo con el método de Galerkin. Graficar la solución.

5. Sea le siguiente problema de valor inicial
$$\begin{cases}\frac{\partial u}{\partial t}=\frac{\partial^2 }{\partial x^2} u,\quad &\text{ en } [0,\pi]\times(0,T]\\\frac{\partial u}{\partial n}(0)=1,\\ u(\pi)=0,\\ u(x,0)=\sin(x),\end{cases}$$ 

 a. Encontrar la formula variacional del problema.

 b. Implementar la aproximación del problema en Fenics y resolverlo con el método de Galerkin. Graficar la solución.

6. Sea le siguiente problema de valor inicial

$$\begin{cases}\frac{\partial u}{\partial t}=\frac{\partial }{\partial x}\Big(x^2\frac{\partial u}{\partial x}\Big),\quad &\text{ en } [0,\pi]\times(0,T]\\u(0)=1,\\ u(1)=0,u(x,0)=\sin(x)\end{cases}$$

a. Encontrar la formula variacional del problema.

b. Implementar la aproximación del problema en Fenics y resolverlo con el método de Galerkin. Graficar la solución.

7. Sea el problema de valor inicial

$$\begin{cases}\frac{\partial u}{\partial t}=\nabla^2 u+f,\quad &\text{ en } \Omega\times(0,T]\\ u(x,y,t)=1 + x^2 + \alpha y^2+ \beta t, &\text{ para }(x,y)\in \partial\Omega\\ u(x,y,0)=1 + x^2 + \alpha y^2\end{cases}$$
donde $\Omega =[0,1]\times[0,1]$, $\alpha = 3,\beta = 1.2$ y $f(x,y)=\beta- 2 - 2\alpha$.

a. Encontrar la formula variacional del problema.


b. Implementar la aproximación del problema en Fenics y resolverlo con el método de Galerkin. Graficar la solución.

8. Sea el problema de valor inicial
$$\begin{cases}\frac{\partial u}{\partial t}=\nabla\cdot\Big(\alpha(u)\nabla(u) \Big) u+f,\quad &\text{ en } \Omega\times(0,T]\\ u(x,y,t)=u_D, &\text{ para }(x,y)\in \partial\Omega\\ u(x,y,0)=u_0\end{cases}$$


a. Encontrar la formula variacional del problema.

b. Escribir un pseudocódigo para resolver el problema.

9. Sea el problema de valor inicial
$$\begin{cases}\frac{\partial u}{\partial t}=\nabla\cdot\Big(\alpha(u)\nabla(u) \Big) u+f,\quad &\text{ en } \Omega\times(0,T]\\ u(x,y,t)=u_D, &\text{ para }(x,y)\in \Gamma_D\\
 \frac{\partial u}{\partial n}=u_N, &\text{ para }(x,y)\in \Gamma_N\\ u(x,y,0)=u_0\end{cases}$$


a. Encontrar la formula variacional del problema.

b. Escribir un pseudocódigo para resolver el problema.